## 코로나 국가별 확진자 수 추이 분석/시각화
- https://public.flourish.studio/visualisation/2897018/

### 날짜별 확진자수 최종 데이터프레임 읽기 (df_confirmed)

In [ ]:
import pandas as pd
df_confirmed = pd.read_csv("COVID-19-master/final_df.csv")
df_confirmed.head()

In [ ]:
df_confirmed.shape

### 국가명과 iso2 매칭 테이블 읽기 (country_info)
  - iso2 컬럼값으로 https://www.countryflags.io/ 에서 제공하는 국기 이미지 링크를 얻을 수 있음
  - 다음 데이터는 결측치로 변환됨
    - ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’
  - Namibia 국가의 iso2 값이 NA 이므로 결측치 변환을 막기 위해 다음과 같은 옵션 설정
    - keep_default_na=False : 디폴트 결측치 변환 데이터를 사용하지 않고, na_values로 지정한 데이터만 결측치로 변환
    - na_values='' : 결측치로 변환할 값을 지정

In [ ]:
country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')
country_info.head()

In [ ]:
country_info[country_info['Country_Region'] == 'Namibia']

In [ ]:
country_info = country_info[['iso2', 'Country_Region']]
country_info.head()

- 중복 행 제거

In [ ]:
country_info.shape

In [ ]:
country_info = country_info.drop_duplicates(subset='Country_Region', keep='last')
country_info.shape

### 날짜별 국가별 확진자수와 국가별 iso2 값 병합

In [ ]:
doc_final_country = pd.merge(df_confirmed, country_info, how='left', on='Country_Region')
doc_final_country.head()

In [ ]:
doc_final_country.shape

#### 없는 데이터(NaN) 확인하기

In [ ]:
doc_final_country.isnull().sum()

### 특정 컬럼에 없는 데이터(NaN)가 있는 경우, 해당 행들만 보여주기
- 특정 컬럼 조건에 맞는 행들만 추출 가능
  - dataframe[dataframe[컬럼명] 조건]

In [ ]:
doc_final_country[doc_final_country['iso2'].isnull()]

#### 참고: 복합 조건은 
- dataframe[(조건1) & (조건2)] : 조건1과 조건2 모두 만족 (and 조건)
- dataframe[(조건1) | (조건2)] : 조건1 또는 조건2 만족 (or 조건)
- 조건에 괄호를 넣는 것이 오동작을 방지하기 위해 좋음

```
nan_rows = doc_final_country[(doc_final_country['iso2'].isnull()) & (doc_final_country['4/01/2020'] == 0)]
nan_rows.head()
```

### 특정 컬럼에 없는 데이터 삭제하기

In [ ]:
doc_final_country = doc_final_country.dropna(subset=['iso2'])

In [ ]:
doc_final_country[doc_final_country['iso2'].isnull()]

### 국기 링크를 기존 컬럼 기반해서 만들어, 데이터프레임에 붙이기
- 필요 데이터
  - 국가명, 국기, 날짜별 확진자 수
- www.countryflags.io
  - AD 는 https://www.countryflags.io/AD/flat/64.png

- 국기 링크 만들기

In [ ]:
iso2= 'AD'
flag_link = 'https://www.countryflags.io/' + iso2 + '/flat/64.png'
flag_link

#### 참고: 데이터프레임에 신규 컬럼 추가하기
- 데이터 프레임에 신규 컬럼 추가하기
  - 데이터프레임[신규컬럼] = pd.Series(data=데이터)
    - 데이터는 행에 맞게, [0, 1, 2] 와 같은 형태로 넣을 수 있음
- 기존 데이터프레임 컬럼 값을 기반으로 신규 컬럼 추가하기
  - apply() 함수를 사용
  - 다음 create_flag_link 함수 적용 전에, 아래의 함수로 어떤 값이 인자로 들어오는지 확인해보기 

    ```
    def create_flag_link(row):
        flag_link = 'https://www.countryflags.io/' + row + '/flat/64.png'
        return flag_link
        
    doc_final_country['Country_Flag'] = doc_final_country['iso2'].apply(create_flag_link)
    ```

In [ ]:
def create_flag_link(row):
    flag_link = 'https://www.countryflags.io/' + row + '/flat/64.png'
    return flag_link

doc_final_country['iso2'] = doc_final_country['iso2'].apply(create_flag_link)

In [ ]:
doc_final_country.head()

### 데이터 포멧 확인하기
<img src="https://www.fun-coding.org/00_Images/covid_ex_data_format.jpg" />

### 데이터프레임 컬럼 조정하기

#### 1. 데이터프레임에서 필요한 컬럼만 선택하기
- 데이터프레임.columns.tolist() : 컬럼명을 리스트로 변환

In [ ]:
cols = doc_final_country.columns.tolist()
cols

#### 데이터프레임 컬럼 위치 변경

In [ ]:
cols.remove('iso2')
cols.insert(1, 'iso2')

In [ ]:
doc_final_country = doc_final_country[cols]

In [ ]:
doc_final_country.head()

#### 컬럼명 변경

In [ ]:
cols[1] = 'Country_Flag'

In [ ]:
doc_final_country.columns = cols
doc_final_country.head()

### 최종 가공 완료 파일 저장

In [ ]:
doc_final_country.to_csv("COVID-19-master/final_covid_data_for_graph.csv")

### 그래프 만들기
- https://app.flourish.studio/ 로그인
- New visualization 선택 -> Bar Chart Race 선택 -> 데이터 수정 (파일 업로드)